# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

In [2]:
df = pd.read_csv("../../data/WDIrevisions.tsv", sep='\t')

### Step 3. Assign it to a variable called chipo.

In [3]:
chipo = df.copy()

### Step 4. See the first 10 entries

In [4]:
chipo.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


### Step 5. What is the number of observations in the dataset?

In [5]:
# Solution 1
len(chipo)

4622

In [6]:
# Solution 2
chipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


### Step 6. What is the number of columns in the dataset?

In [7]:
len(chipo.columns)

5

### Step 7. Print the name of all the columns.

In [8]:
chipo.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

### Step 8. How is the dataset indexed?

In [9]:
chipo.index

RangeIndex(start=0, stop=4622, step=1)

### Step 9. Which was the most-ordered item? 

In [10]:
# 按 item_name 分组，同时对 quantity 和 order_id 进行求和
grouped = chipo.groupby('item_name').agg({
    'order_id': 'sum',  # 对 order_id 列创建一个包含所有值的列表，不过这里的order_id应该是数据给每个商品设计的键，相加可能并没有意义
    'quantity': 'sum'  # 对 quantity 列进行求和
}).reset_index()

# 将 item_name 设置为索引
grouped.set_index('item_name', inplace=True)
# 找出订购数量最多的商品
max_quantity = grouped['quantity'].max()
most_ordered_items = grouped[grouped['quantity'] == max_quantity]

most_ordered_items

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


### Step 10. For the most-ordered item, how many items were ordered?

In [11]:
most_ordered_items

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


### Step 11. What was the most ordered item in the choice_description column?

In [12]:
# 按 choice_description 分组，同时对 quantity 和 order_id 进行求和
grouped = chipo.groupby('choice_description').agg({
    'order_id': 'sum',  # 对 order_id 列创建一个包含所有值的列表，不过这里的 order_id 应该是订单号，order_id 相同说明商品再同一个订单里，这里相加可能没有意义
    'quantity': 'sum'  # 对 quantity 列进行求和
}).reset_index()

# 将 item_name 设置为索引
grouped.set_index('choice_description', inplace=True)
# 找出订购数量最多的商品
max_quantity = grouped['quantity'].max()
most_ordered_items = grouped[grouped['quantity'] == max_quantity]

most_ordered_items

,order_id,quantity
choice_description,,
[Diet Coke],123455,159


### Step 12. How many items were orderd in total?

In [13]:
# 计算 quantity 列的所有值的总和
total_quantity = chipo['quantity'].sum()
int(total_quantity)

4972

### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [14]:
chipo['item_price'].dtype

dtype('O')

#### Step 13.b. Create a lambda function and change the type of item price

In [15]:
# 定义一个函数来清理价格字符串
def clean_price(price):
    if pd.isna(price) or price == '':
        return np.nan  # 返回 NaN 表示缺失值
    if isinstance(price, (int, float)):
        return float(price)  # 如果已经是数字，直接返回
    # 移除美元符号和逗号
    cleaned_price = price.replace('$', '').replace(',', '')
    # 转换为浮点数
    return float(cleaned_price)

#### Step 13.c. Check the item price type

In [16]:
# 应用清理函数到 item_price 列
chipo['item_price'] = chipo['item_price'].apply(clean_price)
chipo['item_price'].dtype

dtype('float64')

### Step 14. How much was the revenue for the period in the dataset?

In [27]:
# 计算每条记录的总价
chipo['total_price'] = chipo['item_price'] * chipo['quantity']
# 计算所有商品的总收益
total_earnings = chipo['total_price'].sum()
print(f"Revenue was: ${total_earnings}")

Revenue was: $39237.02


### Step 15. How many orders were made in the period?

In [19]:
sum_orders = chipo['order_id'].nunique()
sum_orders

1834

### Step 16. What is the average revenue amount per order?

In [28]:
# Solution 1
# 按 order_id 分组并计算每个订单的总价
order_totals = chipo.groupby('order_id')['total_price'].sum().reset_index()
average_order_price = order_totals['total_price'].mean()
print(average_order_price)

21.39423118865867


In [30]:
# Solution 2

# 按 order_id 分组并计算每个订单的总价
order_totals = chipo.groupby('order_id').agg(total_price=('total_price', 'sum')).reset_index()
# 计算所有订单的平均价格
average_order_price = order_totals['total_price'].mean()
print(average_order_price)

21.39423118865867


### Step 17. How many different items are sold?

In [32]:
unique_item_count = chipo['item_name'].nunique()
unique_item_count

50